In [132]:
import pandas as pd
import torch
from torch import nn

In [133]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']
df = pd.read_csv(url, names=column_names, na_values='?', comment='\t', sep=' ', skipinitialspace=True)
df.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [134]:
df.dropna()
df = df.reset_index(drop=True)
print(df.isna().sum())

MPG             0
Cylinders       0
Displacement    0
Horsepower      6
Weight          0
Acceleration    0
Model Year      0
Origin          0
dtype: int64


In [135]:

# train/test split
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, train_size=.8, random_state=1)

train_stats = df_train.describe().transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
MPG,318.0,23.583962,7.890750,9.0,17.50,23.0,29.80,46.6
Cylinders,318.0,5.418239,1.682508,3.0,4.00,4.0,6.00,8.0
Displacement,318.0,191.128931,102.212399,70.0,101.75,146.0,258.00,455.0
Horsepower,312.0,103.317308,37.839804,46.0,75.00,92.0,122.75,230.0
Weight,318.0,2952.047170,836.500568,1613.0,2220.75,2801.0,3533.75,5140.0
Acceleration,318.0,15.717610,2.751966,8.5,14.00,15.5,17.40,24.8
Model Year,318.0,75.921384,3.683467,70.0,73.00,76.0,79.00,82.0
Origin,318.0,1.588050,0.808150,1.0,1.00,1.0,2.00,3.0


In [136]:

numeric_column_names = ['Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration',]

df_train_norm, df_test_norm = df_train.copy(), df_test.copy()

for col_name in numeric_column_names:
    mean = train_stats.loc[col_name, 'mean']
    std = train_stats.loc[col_name, 'std']
    df_train_norm.loc[:, col_name] = (df_train_norm.loc[:, col_name] - mean) / std
    df_test_norm.loc[:, col_name] = (df_test_norm.loc[:, col_name] - mean) / std

df_train_norm.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
393,27.0,-0.842931,-0.500222,-0.457648,-0.193720,-0.042737,82,1
255,25.1,-0.842931,-0.500222,-0.404794,-0.277402,-0.115412,78,1
72,15.0,1.534472,1.104280,1.233693,1.123673,-1.169204,72,1
235,26.0,-0.842931,-0.920915,-0.748347,-0.821335,0.902042,77,3
37,18.0,0.345770,0.399864,-0.087667,0.401617,-0.079074,71,1


In [137]:
boundaries = torch.tensor([73, 76, 79])
v = torch.tensor(df_train_norm['Model Year'].values)
df_train_norm['Model Year Bocketed'] = torch.bucketize(v, boundaries, right=True)
v = torch.tensor(df_test_norm['Model Year'].values)
df_test_norm['Model Year Bocketed'] = torch.bucketize(v, boundaries, right=True)
numeric_column_names.append('Model Year Bocketed')

In [138]:
# one hot encoding
from torch.nn.functional import one_hot

total_origin = len(set(df_train_norm['Origin']))

origin_encoded = one_hot(torch.from_numpy(df_train_norm['Origin'].values) % total_origin)
x_train_numeric = torch.tensor(df_train_norm[numeric_column_names].values)
x_train = torch.cat([x_train_numeric, origin_encoded], 1).float()

origin_encoded = one_hot(torch.from_numpy(df_test_norm['Origin'].values) % total_origin)
x_test_numeric = torch.tensor(df_test_norm[numeric_column_names].values)
x_test = torch.cat([x_test_numeric, origin_encoded], 1).float()


In [139]:
y_train = torch.tensor(df_train_norm['MPG'].values).float()
y_test = torch.tensor(df_test_norm['MPG'].values).float()


In [140]:
from torch.utils.data import TensorDataset, DataLoader

train_ds = TensorDataset(x_train, y_train)
batch_size = 8
torch.manual_seed(1)
train_dl = DataLoader(train_ds, batch_size, shuffle=True)


In [141]:
hidden_units = [8, 4]
input_size = x_train.shape[1]
all_layers = []

for hidden_unit in hidden_units:
    layer = nn.Linear(input_size, hidden_unit)
    all_layers.append(layer)
    all_layers.append(nn.ReLU())
    input_size = hidden_unit

all_layers.append(nn.Linear(hidden_units[-1], 1))
model = nn.Sequential(*all_layers)
model

Sequential(
  (0): Linear(in_features=9, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=4, bias=True)
  (3): ReLU()
  (4): Linear(in_features=4, out_features=1, bias=True)
)

In [142]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)



In [143]:
# train

torch.manual_seed(1)
num_epochs = 200
log_epochs = 20

for epoch in range(num_epochs):
    loss_hist_train = 0
    for x_batch, y_batch in train_dl:
        pred = model(x_batch)[:, 0]
        loss = loss_fn(pred, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        loss_hist_train += loss.item()

    if epoch % log_epochs == 0:
        print(f'Epoch {epoch}  Loss {loss_hist_train/len(train_dl):.4f}')



Epoch 0  Loss nan
Epoch 20  Loss nan
Epoch 40  Loss nan
Epoch 60  Loss nan
Epoch 80  Loss nan
Epoch 100  Loss nan
Epoch 120  Loss nan
Epoch 140  Loss nan
Epoch 160  Loss nan
Epoch 180  Loss nan
